Download dataset

In [15]:
import os
import requests
if not os.path.exists('input.txt'):
    print('Downloading input.txt...')
    url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    r = requests.get(url)

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [17]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

Character based tokenizer, we encode the characters into integers.
We decode the intergers back to characters to get the original text.

In [18]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
# vocab_size
print(f'The vocab size of this tokenizer is {len(stoi)}')
# example
ex = 'You are a douch bag!'
print(f'The encoding is {encode(ex)}')

The vocab size of this tokenizer is 65
The encoding is [37, 53, 59, 1, 39, 56, 43, 1, 39, 1, 42, 53, 59, 41, 46, 1, 40, 39, 45, 2]


Spilt into training and validation set

In [19]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

Next, we randomly sample a batch of text from the training set and use it to train the model. The model will predict the next character in the sequence.   
Since this is simply a classification problem, we will use the predicted character to calculate the cross entropy loss and update the model weights.

In [20]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    # The input (x) is a string of characters.
    # The output (y) is the next character in the string.
    # ix is the random index of a character in the string.
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

We caclulate the average loss over eval_iters

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

The model is only composed of an embedding table.  
![embed.png](images/embed.png)  
idx is a input to the embedding model (embedding table) , its dimension is (B,T) where each element is an integer.  
logits is the output of the model (embedding table), its dimension is (B,T,D) where D is the embedding dimension.  
target is the target of the model, its dimension is (B,T) where each element is an integer.  
The loss is calculated by comparing the logits with the target.  

## generation
We generate token by token, we feed the model with the previous token to get the next token.
We then concatenate the next token to the previous token and feed it back to the model to get the next token.
Since the previous output is fed back to current input, this type of model is called "autoregressive" model.  
![generate.png](images/generate.png)


In [22]:
class LanguageModel(nn.Module):
    '''
    A very simple language model that only looks at the previous one token.
    '''
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size,embedding_dim=vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,D)
        if targets is None:
            loss = None
        else:
            B, T, D = logits.shape
            logits = logits.view(B*T, D)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

Train the model!!!

In [23]:
model = LanguageModel(vocab_size)
m = model.to(device)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step 0: train loss 4.7103, val loss 4.6829
step 300: train loss 2.8052, val loss 2.8130
step 600: train loss 2.5404, val loss 2.5631
step 900: train loss 2.5104, val loss 2.5094
step 1200: train loss 2.4763, val loss 2.5122
step 1500: train loss 2.4777, val loss 2.4960
step 1800: train loss 2.4733, val loss 2.4897
step 2100: train loss 2.4688, val loss 2.4961
step 2400: train loss 2.4632, val loss 2.4895
step 2700: train loss 2.4621, val loss 2.4882


In [25]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



GLENor ath-ld,
BUCA t t t

kind er wodin:

G t Ist h meancof y ht athoup.
CEO rerd trd cis iss od fan IO:
s pifrreroulere noade bulemeis e st stoffendou tinthoubrayod upeae sin t:
Wire ayo neem at d,
Whine jT:
S:
CLe bererdst heat ane bus woryou wingr t, I n:
UEd;
AN t e d b gstos?
O:
Th be sonoune o:
PEENad om tthe tcisonas d chare bidrssupou g fre thyo t y I'ne the d, thaseanckead'dla w pelow y Ty t he, thereeegitos marous thardsisbr ove?

AUS res amit heand k ialanit d, torwe ld ng tce y f l
